In [ ]:
# cd C:\Users\Venuxk\Projects\panel\Data Visualization
# conda activate mypanelintro
# panel serve --show --autoreload elwha.ipynb

from DataVisualizer import DataVisualizer
from ipyleaflet import basemaps
from ipywidgets import Button
import panel as pn
import os
import datetime as dt

app_main_color = "#2196f3"
pn.extension(loading_spinner = "dots", loading_color = app_main_color, sizing_mode = "stretch_width")

In [ ]:
# -------------------------------------------------- Constant Variables --------------------------------------------------

# Set base path to data directories.
base_data_path = "../data/Elwha/"

# Assign names for map's layer types.
topography_data = "Topography"
bathymetry_kayak_data = "Nearshore Bathymetry - Kayak"
bathymetry_watercraft_data = "Nearshore Bathymetry - Personal Watercraft"
grainsize_data = "Surface-Sediment Grain-Size Distributions"
basemap_data = "Basemap"

elwha_data_types = [
  topography_data,
  bathymetry_kayak_data,
  bathymetry_watercraft_data,
  grainsize_data
]

data_type_colors = {
  topography_data: "red",
  bathymetry_kayak_data: "yellow",
  bathymetry_watercraft_data: "green",
  grainsize_data: "#975411"
}

elwha_basemap_options = {
  "Default": basemaps.OpenStreetMap.Mapnik,
  "Satellite": basemaps.Esri.WorldImagery,
  "Topographic": basemaps.OpenTopoMap,
  "Black & White": basemaps.Stamen.Toner,
  "Dark": basemaps.CartoDB.DarkMatter
}

all_latitude_col_names = ["Latitude", "latitude", "Latitude (deg. N)"]
all_longitude_col_names = ["Longitude", "longitude", "Longitude (deg. E)"]
all_datetime_col_names = ["Survey_Date", "datetime_utc", "Time (GMT)", "Time_GMT"]
all_ortho_height_col_names = ["Ortho_Ht_m", "Ortho_ht_m", "ortho_ht_m"]

# -------------------------------------------------- Helper Functions --------------------------------------------------

# Gets the point_style based on the data type.
def get_data_type_point_style(data_type):
  if data_type == topography_data:
    return {"color": "red", "opacity": 0.5, "radius": 0}     # simple dot for topography data since there's many datapoints for one latitude-longitude coordinate
  elif data_type == bathymetry_kayak_data:
    return {"color": "yellow", "radius": 0}
  elif data_type == bathymetry_watercraft_data:
    return {"color": "green", "radius": 0}
  elif data_type == grainsize_data:
    return {"color": "#975411", "opacity": 0.5, "fillColor": "#975411", "fillOpacity": 0.3, "radius": 8, "weight": 1, "dashArray": 2}
  return {}

# Gets the hover_style based on the data type.
def get_data_type_hover_style(data_type):
  if data_type == topography_data:
    return {"color": app_main_color}
  return {"color": app_main_color, "fillColor": app_main_color, "weight": 3}

# Gets the name of an existing dataframe column from the provided list of all possible column names.
def get_existing_col_name(possible_col_names, dataframe_cols):
  for col_name in possible_col_names:
    if col_name in dataframe_cols: return col_name
  return None

# Gets info about the data file and creates a new GeoJSON layer with it.
def create_layer(file, data_type):
  # print("Loading data from " + file + "...")
  # Determine popup content based on different types of data.
  popup_info = {}
  if data_type == grainsize_data:
    popup_info = {
      "Date & Time Collected": [
        "Date Collected",
        [" "],
        {
          "Time (GMT)": "GMT",      # for grainsize data before July 2018
          "Time_GMT": "GMT"         # for grainsize data at and after July 2018
        }
      ],
      "Sample Type": ["Sample Type"],
      "Weight": ["Wt. percent in -2.00 phi bin", ["%"]],
      "Gravel": ["Percent Gravel", ["%"]],
      "Sand": ["Percent Sand", ["%"]],
      "Silt": ["Percent Silt", ["%"]],
      "Clay": ["Percent Clay", ["%"]],
      "Mud": ["Percent Mud", ["%"]]
    }
  elif (data_type == topography_data) or (data_type == bathymetry_kayak_data) or (data_type == bathymetry_watercraft_data):
    popup_info = {
      "Date & Time Collected": [
        {
          "Survey_Date": "",        # for topo-bathy data before July 2018
          "datetime_utc": "UTC"     # for topo-bathy data at and after July 2018
        }
      ],
      "Orthometric Height": [
        {
          "Ortho_Ht_m": "meters",
          "Ortho_ht_m": "meters",
          "ortho_ht_m": "meters"
        },
      ]
    }
  # Create and display GeoJSON layer on map.
  elwha.create_geojson(
    data_path = base_data_path + data_type + "/" + file,
    name = file,
    popup_content = popup_info,
    longitude_col_names = all_longitude_col_names,
    latitude_col_names = all_latitude_col_names
  )

# Checks if the file contains data from the user's selected date range.
def data_within_date_range(filename):
  (selected_start_date, selected_end_date) = data_date_range_slider.value
  
  # Get the data's month and year from its file name.
  month_num = {"jan": 1, "feb": 2, "mar": 3, "apr": 4, "may": 5, "june": 6, "july": 7, "aug": 8, "sept": 9, "oct": 10, "nov": 11, "dec": 12}
  [month_name] = filter(lambda m: m in filename, month_num.keys())
  month = month_num[month_name]
  year = 2000 + int("".join(char for char in filename if char.isdigit()))
  file_date = dt.datetime(year, month, 1)
  
  return selected_start_date <= file_date <= selected_end_date

# -------------------------------------------------- Elwha Topo-Bathy Data Widgets --------------------------------------------------

basemap_select = pn.widgets.Select(name="Basemap", options=list(elwha_basemap_options.keys()))
elwha_data_type_multi_choice = pn.widgets.MultiChoice(name="Type of Data", options=elwha_data_types, placeholder="Choose one or more types of data to display", solid=False)
data_date_range_slider = pn.widgets.DateRangeSlider(
  name = "Data Collection Range",
  start = dt.datetime(2010, 9, 5), end = dt.datetime.utcnow(),
  value = (dt.datetime(2018, 1, 1), dt.datetime(2019, 1, 1)),
  bar_color = app_main_color
)
view_time_series_button = Button(
  description = "View Time-Series",
  button_style = "primary",
  style = dict(button_color = app_main_color)
)

# -------------------------------------------------- Initializing Data Visualization App --------------------------------------------------

# Specify app template.
elwha_data_visualizer = pn.template.BootstrapTemplate(
  site = "Data Visualizer", title = "Elwha Topo-Bathy Data", header_background = app_main_color,
  sidebar = [
    basemap_select,
    elwha_data_type_multi_choice,
    data_date_range_slider
  ]
)

# For all data files, get name and styling information ahead of time so that we just need to modify GeoJSON layers' `data` attribute.
all_data_info = {}
for data_type in elwha_data_types:
  data_type_files = os.listdir(base_data_path + data_type)
  data_type_point_style = get_data_type_point_style(data_type)
  data_type_hover_style = get_data_type_hover_style(data_type)
  for file in data_type_files:
    # Provide optional custom feature styling. Use `all_data_info[file] = {}` to keep ipyleaflet's default feature styling.
    all_data_info[file] = {
      "point_style": data_type_point_style,
      "hover_style": data_type_hover_style
    }

elwha = DataVisualizer(
  data = all_data_info,
  map_center = (48.148, -123.553),
  basemap_options = elwha_basemap_options,
  view_data_button = view_time_series_button,
  legend = {
    "name": "Types of Data",
    "colors": data_type_colors
  }
)

# Add DataVisualizer components to template.
elwha_data_visualizer.main.append(pn.panel(elwha.map))
elwha_data_visualizer.modal.extend(elwha.plotter.results)
# elwha_data_visualizer.modal.append(elwha.plotter.plot)

# -------------------------------------------------- Callbacks & Reactive Functions --------------------------------------------------

# Update basemap whenever a different basemap value is selected.
basemap_select.param.watch(elwha.update_basemap, "value")

# Display time-series plot in a modal whenever the user clicks on the button for viewing how a dataset changes over time.
def plot_time_series(event):
  # Show loading spinner while the scatter plots are being created.
  with pn.param.set_values(elwha_data_visualizer.main[0], loading = True):
    # Plot time-series for all collected data at the hovered/clicked data point's latitude-longitude coordinates.
    [selected_feature_long, selected_feature_lat] = elwha.selected_geojson_data["feature"]["geometry"]["coordinates"]
    elwha.plotter.plot_time_series(
      latitude = selected_feature_lat,
      longitude = selected_feature_long,
      possible_lat_col_names = all_latitude_col_names,
      possible_long_col_names = all_longitude_col_names,
      data_path = "../data/Elwha",
      possible_datetime_col_names = all_datetime_col_names,
      possible_y_axis_col_names = all_ortho_height_col_names,
      data_category_colors = data_type_colors,
      y_axis_label = "Orthometric Height (meters)"
    )
    
    # Plot all data that the hovered/clicked data point was sampled from.
    path = elwha.selected_geojson_data["path"]
    dataframe_cols = elwha.selected_geojson_data["feature"]["properties"].keys()
    [data_color] = [color for data_type, color in data_type_colors.items() if data_type in path]
    elwha.plotter.plot_data(
      data_path = path,
      x_axis_col_name = get_existing_col_name(all_latitude_col_names, dataframe_cols),
      y_axis_col_name = get_existing_col_name(all_longitude_col_names, dataframe_cols),
      data_point_color = data_color
    )

  # Open the app modal to display the scatter plots.
  elwha_data_visualizer.open_modal()

view_time_series_button.on_click(plot_time_series)

# Filters data based on what data type(s) and date range that the user selects.
def filter_data_on_map(event):
  selected_data_types = elwha_data_type_multi_choice.value
  for data_type in elwha_data_types:
    data_type_files = os.listdir(base_data_path + data_type)
    for file in data_type_files:
      if (data_type in selected_data_types) and data_within_date_range(file):
        # Create and display the selected data if we never read the file before.
        if file not in elwha.geojsons:
          # print("create", file)
          create_layer(file, data_type)
        # Display the selected data if it isn't in map yet.
        else:
          # print("display", file)
          elwha.display_geojson(file)
      # Else hide the data if user didn't select to display it.
      else:
        # print("hide", file)
        elwha.hide_geojson(file)

# Filter data whenever the selected data type(s) or date range change.
elwha_data_type_multi_choice.param.watch(filter_data_on_map, "value")
data_date_range_slider.param.watch(filter_data_on_map, "value")

In [ ]:
# Make all the template components servable in a Panel app.
elwha_data_visualizer.servable()